In [1]:
import pandas as pd
import yfinance as yf
import datetime

In [3]:
shorts = ['AAPL', 'TWTR', 'GM', 'INTC', 'AMZN', 'TSLA', 'MSFT', 'GOOGL', 'AMD', 'ACN']
firms = ['Apple', 'Twitter', 'GeneralMotors', 'Intel', 'Amazon', 'Tesla', 'Microsoft', 'Alphabet', 'AMD', 'Accenture']

for short, firm in zip(shorts, firms):
    ticker = yf.Ticker(short)
    history = ticker.history(start='2019-01-01', end='2022-05-03', interval='1d')

    history['average H/L'] = (history['High']+history['Low'])/2
    history = history[['Open', 'High', 'Low', 'Close', 'average H/L']]
    history = history.sort_index(ascending=False)
    history.index.names = ['Date']
    history.reset_index(inplace=True)

    daily_fluc = []

    for i, day in enumerate(history['average H/L']):
        try:
            daily_fluc.append((day/history.loc[i+1, 'average H/L']-1)*100)
        except:
            daily_fluc.append('NA')

    history['daily fluc'] = daily_fluc
    history=history.drop(history.index[-1])

    mean_HT = history['daily fluc'].mean()
    sd_HT = history['daily fluc'].std()
    interval = 1.645*sd_HT/(1**(1/2))
    conf_h = mean_HT+interval
    conf_l = mean_HT-interval
        
    #eliminate irrelevant days
    irrelevant = []

    for i, fluc in enumerate(history['daily fluc']):
        if fluc > conf_h or fluc < conf_l:
            None
        else:
            irrelevant.append(i)

    history = history.drop(history.index[irrelevant])
    
    history.to_csv(f'financial_data/{firm}_financial.csv')
